<a href="https://colab.research.google.com/github/marclagler/MDPI-Project/blob/main/MDPI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn transformers sentence-transformers ngrok poetry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.7/277.7 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.2/331.2 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 453.8/453.8 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 

Load Pre-Trained NLP Model from HuggingFace

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load a pre-trained sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Example usage
sentences = ["Higgs boson in particle physics", "Particle physics at CERN"]
embeddings = model.encode(sentences)

# Check similarity using cosine similarity
cosine_similarity = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
print("Cosine Similarity:", cosine_similarity)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Cosine Similarity: 0.5001519


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
from sentence_transformers import SentenceTransformer

app = FastAPI()
model = SentenceTransformer('all-MiniLM-L6-v2')

class TitleRequest(BaseModel):
    reference: str
    other: list[str]

@app.post("/find_most_similar/")
async def find_most_similar(request: TitleRequest):
    reference_embedding = model.encode([request.reference])[0]
    other_embeddings = model.encode(request.other)

    similarities = [np.dot(reference_embedding, other) /
                    (np.linalg.norm(reference_embedding) * np.linalg.norm(other))
                    for other in other_embeddings]

    best_match_index = np.argmax(similarities)
    return {"top_result": request.other[best_match_index]}


In [ ]:
from pyngrok import ngrok
import uvicorn
import threading

# Run FastAPI using Uvicorn
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run)
thread.start()

# Expose the API publicly
public_url = ngrok.connect(8000).public_url
print("Public URL:", public_url)

INFO:     Started server process [22629]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: https://af2b-35-243-244-212.ngrok-free.app


In [ ]:
import requests

url = "PASTE_PUBLIC_URL_HERE/find_most_similar/"
data = {
    "reference": "Higgs boson in particle physics",
    "other": ["Best soup recipes", "Basel activities", "Particle physics at CERN"]
}

response = requests.post(url, json=data)
print(response.json())

MissingSchema: Invalid URL 'PASTE_PUBLIC_URL_HERE/find_most_similar/': No scheme supplied. Perhaps you meant https://PASTE_PUBLIC_URL_HERE/find_most_similar/?

In [ ]:
#ngrok expires after some time (usually 2 hours for free accounts). If the tunnel stops working, run:
public_url = ngrok.connect(8000).public_url
print("New Public URL:", public_url)